## Project : Weather Pattern Change 

#### Which regions of the US have experienced the largest shifts in weather patterns in recent times?

#### 1. Data source description and ETL

__Data source:__ NOAA <Br>
__Data date range:__ monthly, from 1895 to 2018 <Br>
__Region partition:__ 9 climate regions  <Br>
__Parameters:__ 10 parameters 


In [ ]:
# Package
import plotly.graph_objs as go
import plotly.offline
import pandas as pd
import numpy as np
import requests 
import os

# ETL 
# change work directory
os.chdir('/Users/jiangyur/Downloads/Data')

# Source: www.ncdc.noaa.gov  1895-2018
year_list = list(range(1895, 2019))
month_list = list(range(1, 13))
region_list = region_map.keys()
index_list = [str(yr*100+mo) for yr in year_list for mo in month_list]

# download data through apis
## please only rerun the commented part, otherwise it will take a long time
data = pd.DataFrame(index=index_list)
for rg in region_list:
    for mt in metric_map:
        data_mo = {}
        for mo in month_list:
            try:
                url = 'https://www.ncdc.noaa.gov/cag/regional/time-series/' + str(rg) + '-' + mt + \
                  '-1-' + str(mo) + '-1895-2018.json?base_prd=true&begbaseyear=1895&endbaseyear=2018'
                request = requests.get(url=url, params=None).json()
                data_mo.update(request['data'])
            except:
                continue
        data_mo = pd.DataFrame(data_mo).transpose()
        data_mo.columns = [mt+' '+str(rg), 
                           mt+'_'+'anomaly'+' '+str(rg)]
        data = pd.merge(data, data_mo, how='left', left_index=True, right_index=True)

data.index = pd.to_datetime(data.index, format='%Y%m', errors='ignore')            
data.to_csv('climate_by_region.csv') 
# data = pd.read_csv('climate_by_region.csv', index_col=0, parse_dates=True)
# data.index = pd.to_datetime(data.index, format='%Y%m', errors='ignore')  

In [68]:
# mapping
# region mapping
region_map = {
        101: 'Northeast Climate Region',
        102: 'Upper Midwest Climate Region', 
        103: 'Ohio Valley Climate Region',
        104: 'Southeast Climate Region',        
        105: 'Northern Rockies and Plains Climate Region', 
        106: 'South Climate Region',
        107: 'Southwest Climate Region',        
        108: 'Northwest Climate Region',
        109: 'West Climate Region'
}

# metrics mapping
metric_map = {
        'tavg': 'Average Temperature',
        'tmax': 'Maximum Temperature',
        'tmin': 'Minimum Temperature',
        'cdd': 'Cooling Degree Days',
        'hdd': 'Heating Degree Days',
        'pcp': 'Precipitation',
        'zndx': 'Palmer Z-Index',
        'pdsi': 'Palmer Drought Severity Index (PDSI)',
        'phdi': 'Palmer Hydrological Drought Index (PHDI)',
        'pmdi': 'Palmer Modified Drought Index (PMDI)',
}

#### region map
<img src="https://www.ncdc.noaa.gov/monitoring-references/maps/images/us-climate-regions.gif"> <Br>

#### 2. Quick overview of data through viz

In [71]:
# Take Region 109, Monday, TAVG as an example
region = 109
month = 1  # January
ma = 10  # moving average
yoi = 50  # year of interest
metric = 'tavg'
metric_name = metric+' '+str(region)
metric_ma_name = metric_name+' '+str(ma)

# get data and moving average
data_obj = data.loc[data.index.month==month, [metric_name]].copy()
data_obj[metric_ma_name] = data_obj[metric_name].rolling(ma).mean()
data_obj = data_obj.tail(yoi)   

# viz the chart, a new page will pop up
data_viz = data_obj
trace = go.Candlestick(x=data_viz.index)
data_chart = [trace]
var_list = [metric_name, metric_ma_name]
color_list = ['#17becf', '#7F7F7F']
for i, j in zip(var_list, color_list):    
    data_chart.append(
            go.Scatter(
                    x=data_viz.index.astype('str'),
                    y=data_viz[i],
                    name=str(i),
                    line=dict(color=j),
                    opacity = 0.7))
fig = dict(data=data_chart)
plotly.offline.plot(fig, 'test.html') 

'temp-plot.html'

#### 3. Calculate deviation between most recent 10-year moving average against its 50-year historical baseline

__Baseline mean:__ average(10-year MA between 1969 and 2018) <Br>
__Baseline std:__ standard deviation(10-year MA between 1969 and 2018)<Br>
__Deviation:__ (10-year MA in 2018 - Baseline mean) / Baseline std

In [79]:
# 10-year moving average
ma = 10
yoi = 50
result_diff = {}  
result_std = {}

#  loop through region, month, metric
for region in region_map:
    for month in month_list:
        for metric in metric_map:
            metric_name = metric+' '+str(region)  # e.g. "tvg 101" 
            metric_ma_name = metric_name+' '+str(ma)  # e.g. "101 ma"
            data_obj = data.loc[data.index.month==month, [metric_name]].copy()  # filter by month and metric name
            data_obj[metric_ma_name] = data_obj[metric_name].rolling(ma).mean()  # get 10-year ma for each year-month
            data_obj = data_obj.tail(yoi)  # only retain recent 50 years' 10-year ma-s as baseline
            mean = data_obj[metric_ma_name].mean()
            std = data_obj[metric_ma_name].std()
            diff = data_obj[metric_ma_name].tail(1) - mean
            dev = (data_obj[metric_ma_name].tail(1) - mean) / std
            if region not in result_diff:
                result_diff[region] = {(metric, month): round(diff[0], 2)}
                result_std[region] = {(metric, month): round(dev[0], 2)}
            else:
                result_diff[region].update({(metric, month): round(diff[0], 2)})
                result_std[region].update({(metric, month): round(dev[0], 2)})
                
# transform dictionary to dataframe
data_ma_std = pd.DataFrame(result_std)
data_ma_std.fillna(0, inplace=True)  # NA is introduced by 0/0, indicating no change, fill then with 0. 
data_ma_std['idx'] = data_ma_std.index
data_ma_std['metric'] = data_ma_std['idx'].apply(lambda x: x[0])
data_ma_std['month'] = data_ma_std['idx'].apply(lambda x: x[1])

In [81]:
# sample of result_std
data_ma_std.head()

,,101,102,103,104,105,106,107,108,109,idx,metric,month
tavg,1,0.54,0.48,0.11,0.09,0.92,0.52,1.11,1.39,2.18,"(tavg, 1)",tavg,1
tmax,1,0.44,0.35,0.17,0.17,0.85,0.81,1.31,1.68,2.51,"(tmax, 1)",tmax,1
tmin,1,0.61,0.58,0.07,0.02,0.99,0.16,0.88,1.10,1.57,"(tmin, 1)",tmin,1
cdd,1,0.00,0.00,0.19,0.03,-0.93,0.35,1.97,-1.16,3.23,"(cdd, 1)",cdd,1
hdd,1,-0.40,-0.38,-0.08,-0.12,-0.81,-0.45,-1.30,-1.10,-2.36,"(hdd, 1)",hdd,1


#### 4. Aggregate month into year

__Methodology:__ use a modified square of Euclidean distance calculation to aggregate 12-month into year

In [86]:
# create l2_index: dataframe
l2_index = pd.DataFrame(index=region_map.keys(), columns=metric_map.keys())  
for metric in metric_map:
    data_ma_std_0 = data_ma_std[data_ma_std['metric']==metric]
    for region in region_map:
        l2_index.loc[region, metric] = round(np.mean(np.square(data_ma_std_0[region])), 2)  

l2_index

,tavg,tmax,tmin,cdd,hdd,pcp,zndx,pdsi,phdi,pmdi
101,2.55,2.04,2.79,4.55,2.5,1.38,1.14,0.42,0.55,0.47
102,1.16,0.69,1.65,1.56,1.49,1.66,1.63,3.97,3.29,3.11
103,2.45,1.58,2.79,4.79,2.32,1.35,1.05,2.12,1.74,1.47
104,2.93,2.12,3.39,3.58,2.45,1.4,0.93,0.37,0.32,0.32
105,0.93,0.75,1.33,1.21,1.09,1.85,1.12,0.75,0.86,0.83
106,2.37,1.83,2.71,3.11,1.93,0.85,0.78,1.32,1.27,1.18
107,2.3,2.15,2.41,2.94,2.14,1.26,2.03,1.68,2.21,2.39
108,1.59,1.12,2.3,3.16,1.69,1.99,1.8,0.48,0.24,0.46
109,2.72,2.45,2.57,6.17,3.01,0.52,1.34,4.05,3.73,3.84


#### 5. Unify deviation across parameters

__Methodology:__ Use minmax transformation to keep each parameter within range between 0 and 2. This is because we want to treat each metric (domain) to a same degree.

In [100]:
# minmax transformation
minmax = lambda x, xmax, xmin, a, b: round((x-xmin)*(b-a)/(xmax-xmin)+a, 2)

# unify through minmax
l2_index_norm = l2_index.copy()
for region in region_map:
    for metric in metric_map:
        l2_index_norm.loc[region, metric] = minmax(l2_index.loc[region, metric],
                                                   l2_index[metric].max(), 
                                                   l2_index[metric].min(), 
                                                   0, 
                                                   2)

l2_index_norm        

,tavg,tmax,tmin,cdd,hdd,pcp,zndx,pdsi,phdi,pmdi
101,1.62,1.53,1.42,1.35,1.47,1.17,0.58,0.03,0.18,0.09
102,0.23,0,0.31,0.14,0.42,1.55,1.36,1.96,1.75,1.59
103,1.52,1.01,1.42,1.44,1.28,1.13,0.43,0.95,0.86,0.65
104,2,1.62,2,0.96,1.42,1.2,0.24,0,0.05,0
105,0,0.07,0,0,0,1.81,0.54,0.21,0.36,0.29
106,1.44,1.3,1.34,0.77,0.88,0.45,0,0.52,0.59,0.49
107,1.37,1.66,1.05,0.7,1.09,1.01,2,0.71,1.13,1.18
108,0.66,0.49,0.94,0.79,0.62,2,1.63,0.06,0,0.08
109,1.79,2,1.2,2,2,0,0.9,2,2,2


#### 6. Create final ensemble index

__Methodology:__ Some of the metrics are highly correlated with others. We group them into 3 MOs (Modus Operandi) and will calculate the overall_

In [102]:
# check correlation between metric
corr_df = pd.DataFrame(index=metric_map.keys(), columns=metric_map.keys())
for m1 in metric_map:
    for m2 in metric_map:
        corr_sum = 0
        for region in region_map:
            corr_sum += np.corrcoef(data[m1+' '+str(region)], data[m2+' '+str(region)])[0][1]
        corr_df.loc[m1, m2] = round(corr_sum/len(region_map), 3)
        
corr_df

,tavg,tmax,tmin,cdd,hdd,pcp,zndx,pdsi,phdi,pmdi
tavg,1,0.998,0.997,0.833,-0.982,0.16,-0.039,-0.037,-0.021,-0.031
tmax,0.998,1,0.989,0.823,-0.983,0.123,-0.078,-0.06,-0.039,-0.054
tmin,0.997,0.989,1,0.838,-0.975,0.204,0.007,-0.009,0.001,-0.004
cdd,0.833,0.823,0.838,1,-0.749,0.132,-0.065,-0.062,-0.042,-0.053
hdd,-0.982,-0.983,-0.975,-0.749,1,-0.154,0.03,0.029,0.015,0.024
pcp,0.16,0.123,0.204,0.132,-0.154,1,0.728,0.361,0.267,0.342
zndx,-0.039,-0.078,0.007,-0.065,0.03,0.728,1,0.658,0.537,0.64
pdsi,-0.037,-0.06,-0.009,-0.062,0.029,0.361,0.658,1,0.903,0.944
phdi,-0.021,-0.039,0.001,-0.042,0.015,0.267,0.537,0.903,1,0.966
pmdi,-0.031,-0.054,-0.004,-0.053,0.024,0.342,0.64,0.944,0.966,1


__Comment:__ From the correlation matrix, we can see that there are three groups (MOs):

* __Group1 temperature related:__ tavg, tmax, tmin, cdd, hdd <Br>
* __Group2 precipitation related:__ pcp, zndx <Br>
* __Group3 palmer drought related:__ pdsi, phdi, pmdi <Br>
    
We take __tavg, pcp, pdsi__ as the main metric in each MO, and they are modified to range from 1 to 2. While we take the average of the rest in each MO and modify them into a range from 1 to 1.5. Then we aggregate them with multiplicity to get final change index.

In [101]:
# metric mo dictionary
metric_mo = {
        'temp': [['tavg'], ['tmax', 'tmin', 'cdd', 'hdd']],
        'pcp': [['pcp'], ['zndx']],
        'palmer': [['pdsi'], ['phdi', 'pmdi']]
}

# calculate smooth function: similar concept as sigmoid, smoothly transform x into a value between a and b
smooth = lambda x, a, b: (1/(1+np.e**(-x))-0.5)*(b-a)/0.5 + a

# calculate change_index
l2_index_norm['change_index'] = 0
for region in region_map: 
    # for temp mo:
    temp0 = smooth(l2_index_norm.loc[region, metric_mo['temp'][0]], 1, 2)
    temp1 = smooth(l2_index_norm.loc[region, metric_mo['temp'][1]].mean(), 1, 1.5)
    temp = temp0 * temp1
    # for pcp mo:
    pcp0 = smooth(l2_index_norm.loc[region, metric_mo['pcp'][0]], 1, 2)
    pcp1 = smooth(l2_index_norm.loc[region, metric_mo['pcp'][1]].mean(), 1, 1.5)
    pcp = pcp0 * pcp1
    # for palmer mo:
    palmer0 = smooth(l2_index_norm.loc[region, metric_mo['palmer'][0]], 1, 2)
    palmer1 = smooth(l2_index_norm.loc[region, metric_mo['palmer'][1]].mean(), 1, 1.5)
    palmer = palmer0 * palmer1
    # aggregate with multiplication 
    l2_index_norm.loc[region, 'change_index'] = round(temp[0] * pcp[0] * palmer[0], 3)

# see change_index in descending order
l2_index_norm.sort_values(by='change_index', ascending=False)

,tavg,tmax,tmin,cdd,hdd,pcp,zndx,pdsi,phdi,pmdi,change_index
109,1.79,2,1.2,2,2,0,0.9,2,2,2,6.856
107,1.37,1.66,1.05,0.7,1.09,1.01,2,0.71,1.13,1.18,6.846
103,1.52,1.01,1.42,1.44,1.28,1.13,0.43,0.95,0.86,0.65,5.995
102,0.23,0,0.31,0.14,0.42,1.55,1.36,1.96,1.75,1.59,5.907
101,1.62,1.53,1.42,1.35,1.47,1.17,0.58,0.03,0.18,0.09,3.993
104,2,1.62,2,0.96,1.42,1.2,0.24,0,0.05,0,3.804
108,0.66,0.49,0.94,0.79,0.62,2,1.63,0.06,0,0.08,3.779
106,1.44,1.3,1.34,0.77,0.88,0.45,0,0.52,0.59,0.49,3.490
105,0,0.07,0,0,0,1.81,0.54,0.21,0.36,0.29,2.332


#### Finding: region 109 and region 107 have experienced the most changes in recent times. 

* __West Climate Region (109)__: Dramatic change in temperature and drought.<Br>
    
* __Southwest Climate Region (107)__: Significant changes across all indicators.<Br>

This makes a lot sense as 109 and 107 are next to each other.<Br>

(Also, 102 and 103 are also next to each other, which makes the analysis even more convincing)

In [106]:
l2_index_norm.loc[[107,109], :].sort_values(by='change_index', ascending=False)

,tavg,tmax,tmin,cdd,hdd,pcp,zndx,pdsi,phdi,pmdi,change_index
109,1.79,2,1.2,2,2,0,0.9,2,2,2,6.856
107,1.37,1.66,1.05,0.7,1.09,1.01,2,0.71,1.13,1.18,6.846


------------------------------------------------------------- end -------------------------------------------------------------